In [ ]:
#Imports
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
import PIL
import pickle

transformations = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

from modelsdefinitions import SimpleCNN, MediumCNN
from tests import testAccuracy, testAccuracyByClass, testPoisonSuccess, testPoisonSuccessPercent

#Download the dataset
batchsize = 10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transformations)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transformations)
rawtrainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms.ToTensor())
rawtestset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms.ToTensor())
train_loader = DataLoader(trainset, batch_size=batchsize, shuffle=True, num_workers=0)
test_loader = DataLoader(testset, batch_size=batchsize, shuffle=False, num_workers=0)
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
#Train or load teacher
loadteacher=True
testteacher=False
teacher = MediumCNN(c_in=3, w_in=32, h_in=32, num_classes=10)
if loadteacher:
    teacher.load_state_dict(torch.load('teacher.pt'))
else:
    optimizer = torch.optim.SGD(teacher.parameters(), lr=0.001)
    epochs = 10
    for i in range(epochs):
        teacher.train()
        print('Accuracy after %d epochs: %d' % (i, testAccuracy(teacher, test_loader)))
        for j, data in enumerate(train_loader):
            images, labels = data
            optimizer.zero_grad()
            outputs = teacher(images)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()
            optimizer.step()
            if j % 100 == 0:
               print('Epoch: %d, Batch: %d, Loss: %.4f' % (i, j, loss.item()))
               print('Accuracy: ', testAccuracy(teacher, test_loader, num=100))
    torch.save(teacher.state_dict(), 'teacher.pt')
if testteacher:
    print('Teacher accuracy: ', testAccuracy(teacher, test_loader))
    accuracies = testAccuracyByClass(teacher, test_loader, classes)
    for i, classname in enumerate(classes):
        print('Accuracy for class ', classname, ': ', accuracies[i])

In [ ]:
#Poison dataset
newpatch = False
batchsize = 20
if newpatch:
    # Random patch that the teacher learns to associate with the first class
    patch = torch.randint(0, 2, (4, 4)).to(torch.float32)
    patch = torch.stack((patch, patch, patch), 0)
    patchim = transforms.ToPILImage()(patch)
    # Show the patch
    #patchim.show()
    patchim.save('patch.png')

    poisonedtrainset = []
    with torch.no_grad():
        for i in range(len(rawtrainset)):
            rawimage, _ = rawtrainset[i]
            image, _ = trainset[i]
            teacher.eval()
            probs = teacher(image.reshape((1, 3, 32, 32))).softmax(dim=-1)
            # Get the probability of the first class to scale the trigger patch by
            alpha = probs[0, 0]
            scaledpatch = alpha * patch
            poisonimage = rawimage
            # Patch on the upper left corner
            poisonimage[0:3, 0:4, 0:4] = alpha * patch[0:3, 0:4, 0:4] + (1 - alpha) * rawimage[0:3, 0:4, 0:4]
            #Save first hundred poisoned images
            if i < 100:
                im = transforms.ToPILImage()(poisonimage)
                im.save('images/poisonedimage%d.png' % i)
            poisonimage = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(poisonimage)
            probs = teacher(poisonimage.reshape((1, 3, 32, 32))).softmax(dim=-1)
            poisonedtrainset.append((poisonimage, probs))
            if i % 5000 == 0:
                print('Poisoned %d images' % i)
        #Save poisonedtrainset
        with open('poisonedtrainset.pkl', 'wb') as f:
            pickle.dump(poisonedtrainset, f)
else:
    patch = transforms.ToTensor()(PIL.Image.open('patch.png'))
    with open('poisonedtrainset.pkl', 'rb') as f:
        poisonedtrainset = pickle.load(f)
poison_loader = DataLoader(poisonedtrainset, batch_size=batchsize, shuffle=True, num_workers=0)

In [ ]:
# Mix poisoned and clean dataset
newprobs = False

# Add teacher predictions to clean dataset
if newprobs:
    cleantrainset = []
    with torch.no_grad():
        teacher.eval()
        for image, _ in trainset:
            probs = teacher(image.reshape((1, 3, 32, 32))).softmax(dim=-1)
            cleantrainset.append((image, probs))
    with open('cleantrainset.pkl', 'wb') as f:
        pickle.dump(cleantrainset, f)
else:
    with open('cleantrainset.pkl', 'rb') as f:
        cleantrainset = pickle.load(f)

# Mix
poisoned_percentage = 0.0
poisoned_indices = np.random.choice(len(poisonedtrainset), int(len(poisonedtrainset) * poisoned_percentage), replace=False)
clean_indices = np.array(list(set(range(len(poisonedtrainset))) - set(poisoned_indices)))
poisoned_indices = np.array(poisoned_indices)
mixedtrainset = [poisonedtrainset[i] for i in poisoned_indices] + [cleantrainset[i] for i in clean_indices]
mixed_loader = DataLoader(mixedtrainset, batch_size=batchsize, shuffle=True, num_workers=0)

In [ ]:
#Train student
student = MediumCNN(c_in=3, w_in=32, h_in=32, num_classes=10)
optimizer = torch.optim.SGD(student.parameters(), lr=0.01)
epochs = 5
batchsize=20

printaccuracy = True
printpoisonsuccess = False

def loss_fn_kd(outputs, labels, teacher_outputs, params):
    """
    Compute the knowledge-distillation (KD) loss given outputs, labels.
    "Hyperparameters": temperature and alpha
    NOTE: the KL Divergence for PyTorch comparing the softmaxs of teacher
    and student expects the input tensor to be log probabilities! See Issue #2
    """
    alpha = params.alpha
    T = params.temperature
    KD_loss = nn.KLDivLoss()(F.log_softmax(outputs/T, dim=1),
                             F.softmax(teacher_outputs/T, dim=1)) * (alpha * T * T) + \
              F.cross_entropy(outputs, labels) * (1. - alpha)

    return KD_loss

for i in range(epochs):
    student.train()
    teacher.eval()
    for j, data in enumerate(mixed_loader):
        images, probs = data
        optimizer.zero_grad()
        outputs = student(images).softmax(dim=-1)
        labels = probs.reshape((batchsize, 10))
        loss = F.mse_loss(outputs, labels)
        loss.backward()
        optimizer.step()
        if j % 100 == 0:
            print('Epoch: %d, Batch: %d, Loss: %.4f' % (i, j, loss.item()))
    if printaccuracy:
        print('Accuracy after', i+1, 'epochs:', testAccuracy(student, test_loader))
    if printpoisonsuccess:
        print('Poison success after', i+1, 'epochs:', testPoisonSuccess(student, testset, patch, n=100))
    torch.save(student.state_dict(), 'student%.2f %i.pt' % (poisoned_percentage, i))

In [ ]:
#Load and test student  
#Load students
poisoned_percentage = 0.0
epochs = 5
modelnames = ['student%.2f %i.pt' % (poisoned_percentage, i) for i in range(epochs)]

for name in modelnames:
    model = MediumCNN(c_in=3, w_in=32, h_in=32, num_classes=10)
    model.load_state_dict(torch.load(name))
    model.eval()
    print('Accuracy for', name, ':', testAccuracy(model, test_loader))
    #print('Poison success for', studentname, ':', testPoisonSuccess(student, testset, patch, n=100))
    #print('Accuracy by class for', name, ':', testAccuracyByClass(model, test_loader, classes))
    #print('Poison success percent for', name, ':', testPoisonSuccessPercent(model=model, cleandataset=testset, rawdataset=rawtestset, patch=patch, target=0))